In [6]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


я хочу сделать функцию, которая будет принимать на вход 2 пары коодинат : одна - координата страны, которые есть в моей первой части кода, вторая пара - координаты склада из кода Полины (мб пары наоборот)

ПЛАН
- ввести в поиск одни координаты
- нажать на маршрут
- ввести вторые координаты
- получить расстояние между ними


In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time

In [10]:
def get_distance(lat1, lon1, lat2, lon2):
    driver = webdriver.Chrome()
    driver.get("https://yandex.ru/maps")
    #жду до условия
    try:
        my_search = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "input__control"))) #чекнуть, что класс, там еще bold, оно не оно
        my_search.send_keys(str(lat1), ' ', str(lon1))
        my_search.send_keys(Keys.RETURN) #имитирую нажание enter с клавиатуры
    
        #ввели координаты, теперь нужно, чтобы появилось окошко для вторых координат, хочу нажать на маршрутную кнопку
        my_path = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "action-button-view._type_route")))
        my_path.click()
        #теперь я оказалась в месте, где заполнено "куда" и хочу попасть в "откуда"
        
        #так же как заполняла search заполняю "откуда"
        time.sleep(2)
        froms= WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "input__control")))
        time.sleep(2)

        my_from = froms[1]
        my_from.send_keys(str(lat2), ' ', str(lon2))
        my_from.send_keys(Keys.RETURN) 
        length = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "auto-route-snippet-view__route-subtitle"))).text
        return length
    except:
        print('ошибка в момент обращения')
    finally:
        driver.quit()

#lat1, lon1 - координаты точки назначения
#lat2, lon2 - координаты точки отправления


In [ ]:
pip install pandas

In [1]:
import pandas as pd

In [ ]:
#я скачала итоговый фрейм из коллаба, там лежат координаты стран
path = "C:/Users/User/Downloads/final_df.csv"
df = pd.read_csv(path)
#выкидываю, все, что не нужно
df = df[['Country name', 'latitude', 'longitude']]
print(df)

      Country name  latitude  longitude
0       Kyrgyzstan   42.8670    74.6000
1          Belarus   53.9170    27.5500
2       Tajikistan   38.5500    68.8000
3          Armenia   40.1830    44.5170
4         Bulgaria   42.6975    23.3225
5       Azerbaijan   40.3830    49.8330
6          Albania   41.3170    19.8170
7  North Macedonia   42.0000    21.4330
8          Georgia   41.7170    44.7830


In [ ]:
#ищу расстояния между складом в казани и каждой из стран
df['Расстояние от склада до страны'] = None
lat_kaz = float(55.79083)
lon_kaz = float(49.11444) #координаты склада
for index, row in df.iterrows():
    lat_country = row['latitude']
    lon_country = row['longitude']
    factory_country = get_distance(lat_country, lon_country, lat_kaz, lon_kaz)
    print(factory_country) #это не обязательно, но все долго грузится и я проверяю, что нигде ничего не умерло
    df.at[index, 'Расстояние от склада до страны'] = factory_country
print(df)

3 000 км
1 550 км
3 100 км
2 310 км
3 400 км
ошибка в момент обращения
None
3 600 км
3 500 км
2 040 км
      Country name  latitude  longitude Расстояние от завода до страны
0       Kyrgyzstan   42.8670    74.6000                       3 000 км
1          Belarus   53.9170    27.5500                       1 550 км
2       Tajikistan   38.5500    68.8000                       3 100 км
3          Armenia   40.1830    44.5170                       2 310 км
4         Bulgaria   42.6975    23.3225                       3 400 км
5       Azerbaijan   40.3830    49.8330                           None
6          Albania   41.3170    19.8170                       3 600 км
7  North Macedonia   42.0000    21.4330                       3 500 км
8          Georgia   41.7170    44.7830                       2 040 км


In [ ]:
#теперь выбираю страны, в которые мы все же поставляем
df_sorted = df.sort_values(by='Расстояние от склада до страны')
df_top5 = df_sorted.head(6)
print(df_top5)

  Country name  latitude  longitude Расстояние от завода до страны
1      Belarus   53.9170    27.5500                       1 550 км
8      Georgia   41.7170    44.7830                       2 040 км
3      Armenia   40.1830    44.5170                       2 310 км
0   Kyrgyzstan   42.8670    74.6000                       3 000 км
2   Tajikistan   38.5500    68.8000                       3 100 км
4     Bulgaria   42.6975    23.3225                       3 400 км


In [ ]:
#теперь ищу для каждого сорта растения расстояние от поля до склада, result_df - датасет, который скачала из ноутбука Полины
path2 = "C:/Users/User/Downloads/result_df.csv"
df_polina = pd.read_csv(path2)
df_polina = df_polina.drop(0)
print(df_polina)


   Unnamed: 0            город   долгота    широта  растение цена
1           0  Санкт-Петербург  30.31667  59.95000   мелисса  100
2           1          Чусовой  57.81667  58.28333  иван-чай  250
3           2           Тюмень  65.53333  57.15000      мята  100


In [ ]:
df_polina['Расстояние от растения до склада'] = None
lat_kaz = float(55.79083)
lon_kaz = float(49.11444) #координаты склада
for index, row in df_polina.iterrows():
    lat_plant = row['широта']
    lon_plant = row['долгота']
    factory_plant = get_distance(lat_kaz, lon_kaz, lat_plant, lon_plant)
    print(factory_plant) 
    df_polina.at[index, 'Расстояние от растения до склада'] = factory_plant
print(df_polina)

1 550 км
840 км
1 360 км
   Unnamed: 0            город   долгота    широта  растение цена  \
1           0  Санкт-Петербург  30.31667  59.95000   мелисса  100   
2           1          Чусовой  57.81667  58.28333  иван-чай  250   
3           2           Тюмень  65.53333  57.15000      мята  100   

  Расстояние от растения до завода  
1                         1 550 км  
2                           840 км  
3                         1 360 км  
